In [1]:
import requests
import json

import pandas as pd
from pandas.tseries.frequencies import to_offset

import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math

from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

import time
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
# from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
segment_size = 100

# 1727 rows
data = pd.read_csv('../Dataset/STB.csv')

# Calculate the number of segments in the dataset
print("len(data)", len(data))
# num_segments = int(len(data)/segment_size)
# # Đọc vào tập dữ liệu bị phân mảnh ngang
# data_chunks = []
# for i in range(num_segments):
#     chunk = pd.read_csv(f'../Dataset/STB.csv')
#     data_chunks.append(chunk)
# data = pd.concat(data_chunks, ignore_index=True)
# data['date'] = pd.to_datetime(data.trunc_time)
# Áp dụng moving average với cửa sổ 5 và lưu vào cột 'ma_5'
# data['ma_5'] = data['close_price'].rolling(window=1,min_periods=1).mean()
data['ma_5'] = data['close_price'].ewm(span=3, adjust=False, min_periods=0).mean()

# Xóa bỏ các dòng có giá trị null do moving average
data.dropna(inplace=True)
# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
# train_data = data.sample(frac=0.8, random_state=42)
# test_data = data.drop(train_data.index)

len(data) 1727


In [3]:
# closedf = data[['trunc_time','ma_5']]
closedf = data[['trunc_time','close_price','ma_5']]
print(closedf)
print("Shape of close dataframe:", closedf.shape)

      trunc_time  close_price          ma_5
0     2016-01-04        12600  12600.000000
1     2016-01-05        12300  12450.000000
2     2016-01-06        12800  12625.000000
3     2016-01-07        12600  12612.500000
4     2016-01-08        12600  12606.250000
...          ...          ...           ...
1722  2022-11-21        16700  16748.526427
1723  2022-11-22        16900  16824.263213
1724  2022-11-23        17500  17162.131607
1725  2022-11-24        18200  17681.065803
1726  2022-11-25        18900  18290.532902

[1727 rows x 3 columns]
Shape of close dataframe: (1727, 3)


In [4]:
# close_stock = closedf.copy()
# del closedf['trunc_time']
# scaler=MinMaxScaler(feature_range=(0,1))
# closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
# print(closedf.shape)

In [5]:
import copy

close_stock = closedf.copy()
del closedf['trunc_time']

X_data = copy.deepcopy(closedf)
Y_data = copy.deepcopy(closedf)

del X_data['close_price']
del Y_data['ma_5']

scaler=MinMaxScaler(feature_range=(0,1))

X_data=scaler.fit_transform(np.array(X_data).reshape(-1,1))
Y_data=scaler.fit_transform(np.array(Y_data).reshape(-1,1))

print(X_data, X_data.shape)
print(Y_data, Y_data.shape)

[[0.18182456]
 [0.17650269]
 [0.18271154]
 ...
 [0.34368504]
 [0.36209637]
 [0.38371974]] (1727, 1)
[[0.18563923]
 [0.17513135]
 [0.19264448]
 ...
 [0.35726795]
 [0.38178634]
 [0.40630473]] (1727, 1)


In [6]:
del closedf['close_price']
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))

print(closedf.shape)

(1727, 1)


In [7]:
training_size=int(len(closedf)*0.8)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)
# train_data

train_data:  (1381, 1)
test_data:  (346, 1)


In [8]:
# convert an array of values into a dataset matrix
# def create_dataset(dataset, time_step=1):
#     dataX, dataY = [], []
#     for i in range(len(dataset)-time_step-1):
#         a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
#         dataX.append(a)
#         dataY.append(dataset[i + time_step, 0])
#     return np.array(dataX), np.array(dataY)
def create_dataset(data_set):
    training_size=int(len(data_set)*0.8)
    # test_size=len(data_set)-training_size
    train_data,test_data=data_set[0:training_size,:],data_set[training_size:len(data_set),:1]
    print("train_data: ", train_data.shape)
    print("test_data: ", test_data.shape)
    return train_data,test_data

In [9]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 10
# print(type(X_data))
# X_train, y_train = create_dataset(train_data, time_step)
# X_test, y_test = create_dataset(test_data, time_step)
X_train, X_test = create_dataset(X_data)
y_train, y_test = create_dataset(Y_data)
print("X_train: ", X_train)
print("y_train: ", y_train)
print("X_test: ", X_test)
print("y_test", y_test)
# print("type(X_test)",type(X_test))

train_data:  (1381, 1)
test_data:  (346, 1)
train_data:  (1381, 1)
test_data:  (346, 1)
X_train:  [[0.18182456]
 [0.17650269]
 [0.18271154]
 ...
 [0.82183932]
 [0.79453479]
 [0.76846483]]
y_train:  [[0.18563923]
 [0.17513135]
 [0.19264448]
 ...
 [0.80210158]
 [0.76357268]
 [0.73905429]]
X_test:  [[0.76075172]
 [0.73383373]
 [0.74521013]
 [0.74468948]
 [0.72757657]
 [0.72966385]
 [0.7262726 ]
 [0.72723791]
 [0.74013826]
 [0.74658844]
 [0.75690935]
 [0.7585219 ]
 [0.766424  ]
 [0.78101878]
 [0.77767244]
 [0.78486905]
 [0.80354599]
 [0.81998028]
 [0.81489276]
 [0.81944482]
 [0.81994689]
 [0.81665002]
 [0.80080993]
 [0.79466384]
 [0.80223453]
 [0.80335895]
 [0.80037324]
 [0.79888038]
 [0.76886367]
 [0.73079388]
 [0.71619387]
 [0.71066783]
 [0.70346991]
 [0.70341887]
 [0.71403709]
 [0.70958944]
 [0.70470467]
 [0.70847114]
 [0.71656322]
 [0.70996553]
 [0.70755365]
 [0.70368678]
 [0.69643148]
 [0.68836893]
 [0.68345068]
 [0.68365249]
 [0.68464038]
 [0.69045619]
 [0.68804222]
 [0.68506128]
 [0

In [24]:
class Node:
    def __init__(self, split_feature=None, split_value=None, left=None, right=None, prediction=None, is_leaf=False):
        self.split_feature = split_feature
        self.split_value = split_value
        self.left = left
        self.right = right
        self.prediction = prediction
        self.is_leaf = is_leaf

In [25]:
import numpy as np

class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _split(self, X, y):
        best_feature = None
        best_value = None
        best_score = float('inf')

        m, p = X.shape
        features = np.random.choice(p, self.max_features, replace=False) if self.max_features else range(p)

        for feature in features:
            values = np.unique(X[:, feature])
            if len(values) <= 1:
                continue

            for i in range(len(values) - 1):
                value = (values[i] + values[i+1]) / 2

                left_indices = X[:, feature] <= value
                right_indices = X[:, feature] > value

                if len(y[left_indices]) < self.min_samples_leaf or len(y[right_indices]) < self.min_samples_leaf:
                    continue

                score = len(y[left_indices]) * np.var(y[left_indices]) + len(y[right_indices]) * np.var(y[right_indices])

                if score < best_score:
                    best_feature = feature
                    best_value = value
                    best_score = score

        return best_feature, best_value, best_score

    def _build_tree(self, X, y, node=None, depth=0):
        if node is None:
            node = Node()

        if depth == self.max_depth or len(y) < self.min_samples_split:
            node.is_leaf = True
            node.prediction = np.mean(y)
            return node

        split_feature, split_value, split_score = self._split(X, y)

        if split_feature is None:
            node.is_leaf = True
            node.prediction = np.mean(y)
            return node

        left_indices = X[:, split_feature] <= split_value
        right_indices = X[:, split_feature] > split_value

        node.split_feature = split_feature
        node.split_value = split_value

        node.left = self._build_tree(X[left_indices], y[left_indices], Node(), depth + 1)
        node.right = self._build_tree(X[right_indices], y[right_indices], Node(), depth + 1)

        return node

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])
    
    def _traverse_tree(self, x, node):
        if node.is_leaf:
            return node.prediction
        else:
            if x[node.split_feature] <= node.split_value:
                return self._traverse_tree(x, node.left)
            else:
                return self._traverse_tree(x, node.right)

In [26]:
import numpy as np

class RandomForestRegressor:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.trees = []
        
    def fit(self, X, y):
        for i in range(self.n_estimators):
            bootstrap_indices = np.random.choice(len(X), size=len(X), replace=True)
            bootstrap_X = X[bootstrap_indices]
            bootstrap_y = y[bootstrap_indices]
            
            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split, 
                                          min_samples_leaf=self.min_samples_leaf, max_features=self.max_features)
            tree.fit(bootstrap_X, bootstrap_y)
            self.trees.append(tree)
            
    def predict(self, X):
        return np.mean([tree.predict(X) for tree in self.trees], axis=0)

In [38]:
from sklearn.metrics import mean_squared_error
import optuna
# from sklearn.ensemble import RandomForestRegressor as SklearnRandom

# Define the objective function for Optuna
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_int("max_features", 1, X_train.shape[1])

    # max_features = trial.suggest_float("max_features", 0.1, 1.0)
    # max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt']) \
    #             if trial.suggest_categorical('max_features', ['auto', 'sqrt']) else \
    #             trial.suggest_int('max_features', 1, X_train.shape[1])
    # max_features_fraction = trial.suggest_categorical('max_features_fraction', ['auto', 'sqrt', 0.5, 0.75, 1.0])

    # if max_features_fraction == 'auto' or max_features_fraction == 'sqrt':
    #     max_features = max_features_fraction
    # else:
    #     max_features = max(int(max_features_fraction * X_train.shape[1]), 1)
    print(n_estimators)
    max_features = int(max_features * X_train.shape[1])
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features    
    )

    print(n_estimators)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse

# Use Optuna to optimize hyperparameters
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=6, timeout = 600)
best_params = study.best_params

rf = RandomForestRegressor(**best_params)


[I 2023-05-08 17:29:04,320] A new study created in memory with name: no-name-951fe3f6-5fb3-45ed-95a9-3719277a07ae


715
715


[I 2023-05-08 17:47:41,285] Trial 0 finished with value: 0.0009051196806188548 and parameters: {'n_estimators': 715, 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 1}. Best is trial 0 with value: 0.0009051196806188548.


In [39]:
# rf = RandomForestRegressor(max_depth=26, min_samples_leaf=2, min_samples_split=10,n_estimators=160, max_features=1)
# # rf.fit(X_train, y_train)

In [40]:
# rf = SklearnRandom(max_depth=26, min_samples_leaf=2, min_samples_split=10,
#                       n_estimators=160)
rf.fit(X_train, y_train)

In [41]:
# Lets Do the prediction 

RF_train_predict=rf.predict(X_train)
RF_test_predict=rf.predict(X_test)
# print("Train data prediction:", train_predict)
# # print("Test data prediction:", test_predict)
RF_train_predict = RF_train_predict.reshape(-1,1)
RF_test_predict = RF_test_predict.reshape(-1,1)

print("Train data prediction:", RF_train_predict.shape)
print("Test data prediction:", RF_test_predict.shape)

Train data prediction: (1381, 1)
Test data prediction: (346, 1)


In [42]:
# Transform back to original form

RF_train_predict = scaler.inverse_transform(RF_train_predict)
RF_test_predict = scaler.inverse_transform(RF_test_predict)
RF_original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
RF_original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [43]:
# Evaluation metrices RMSE and MAE
RF_RMSE_train = math.sqrt(mean_squared_error(RF_original_ytrain,RF_train_predict))
RF_MSE_train = mean_squared_error(RF_original_ytrain,RF_train_predict)
RF_MAE_train = mean_absolute_error(RF_original_ytrain,RF_train_predict)

RF_RMSE_test = math.sqrt(mean_squared_error(RF_original_ytest,RF_test_predict))
RF_MSE_test = mean_squared_error(RF_original_ytest,RF_test_predict)
RF_MAE_test = mean_absolute_error(RF_original_ytest,RF_test_predict)

print("Train data RMSE: ", RF_RMSE_train)
print("Train data MSE: ", RF_MSE_train)
print("Test data MAE: ", RF_MAE_train)
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", RF_RMSE_test)
print("Test data MSE: ", RF_MSE_test)
print("Test data MAE: ", RF_MAE_test)

Train data RMSE:  121.26654384599958
Train data MSE:  14705.57465635374
Test data MAE:  78.3358498913885
-------------------------------------------------------------------------------------
Test data RMSE:  849.3215079762628
Test data MSE:  721347.023911073
Test data MAE:  642.2601479597051


In [44]:
RF_EV_train = explained_variance_score(RF_original_ytrain, RF_train_predict)
RF_EV_test = explained_variance_score(RF_original_ytest, RF_test_predict)

print("Train data explained variance regression score:", RF_EV_train)
print("Test data explained variance regression score:", RF_EV_test)

Train data explained variance regression score: 0.9991103892523654
Test data explained variance regression score: 0.9728988543711831


In [45]:
RF_r2_train = r2_score(RF_original_ytrain, RF_train_predict)
RF_r2_test = r2_score(RF_original_ytest, RF_test_predict)

print("Train data R2 score:", RF_r2_train)
print("Test data R2 score:", RF_r2_test)

Train data R2 score: 0.999110388882773
Test data R2 score: 0.9716293923346284


In [46]:
RF_MGD_train = mean_gamma_deviance(RF_original_ytrain, RF_train_predict)
RF_MGD_test = mean_gamma_deviance(RF_original_ytest, RF_test_predict)
RF_MPD_train = mean_poisson_deviance(RF_original_ytrain, RF_train_predict)
RF_MPD_test = mean_poisson_deviance(RF_original_ytest, RF_test_predict)
print("Train data MGD: ", RF_MGD_train)
print("Test data MGD: ", RF_MGD_test)
print("----------------------------------------------------------------------")
print("Train data MPD: ", RF_MPD_train)
print("Test data MPD: ",RF_MPD_test)

Train data MGD:  6.779803924639389e-05
Test data MGD:  0.0009255636936139461
----------------------------------------------------------------------
Train data MPD:  0.9317346993993997
Test data MPD:  25.339358417872358


In [47]:
# shift train predictions for plotting
from itertools import cycle
import plotly.express as px

print(RF_train_predict.shape)
print(RF_test_predict.shape)
print(closedf.shape)
print(len(close_stock))

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
# trainPredictPlot[look_back:len(RF_train_predict)+look_back, :] = RF_train_predict
trainPredictPlot[0:len(RF_train_predict), :] = RF_train_predict

print("Train predicted data: ", trainPredictPlot)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
# testPredictPlot[len(RF_train_predict)+(look_back*2)+1:len(closedf)-1, :] = RF_test_predict
testPredictPlot[len(RF_train_predict):len(closedf), :] = RF_test_predict

print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['trunc_time'],
                       'original_close': close_stock['ma_5'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

(1381, 1)
(346, 1)
(1727, 1)
1727
Train predicted data:  [[12720.40061951]
 [12447.39418026]
 [12904.8870558 ]
 ...
 [           nan]
 [           nan]
 [           nan]]
Test predicted data:  (1727, 1)


In [48]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
#         print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = rf.predict(x_input)
#         print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = rf.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


In [49]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10]
[11 12 13 14 15 16 17 18 19 20]


In [50]:
import copy

temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = copy.deepcopy(temp_mat)
next_predicted_days_value = copy.deepcopy(temp_mat)

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]
last_original_days_value.append(None)
names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()